# Image processing with OpenCV
## Margin Removal
***Objective:*** *to crop the (black) background from page images, only leaving the minimum margin outside the main image of
the page itself. Also produces a binary (1-bit) image suitable for feeding into an OCR/HTR pipeline.
The algorithm as it stands does need a fairly dark, even background (e.g. the grey copy stand with grid causes problems),
though the threshold parameters could be adjusted to accommodate this.*

**Repository:** https://github.com/gbstringer/python-improc/

#### Import numpy, scikit-image and open computer vision libraries
* **numpy** (*) provides the matrix arithmetic (not currently used)
* **cv2** (opencv-python) provides the image processing functions
* **skimage** (scikit-image) provides the embedded image display in Jupyter
* **matplotlib** (*) is used to lay out side-by-side comparisons (not currently used)
* **copy** is needed to perform deep copies of the images (alternatives?)

These packages (*) will need to be installed in the virtual instance or globally if running from Jupyter itself. You may
also need to install Qt packages for the interactive image display.

In [1]:
#from matplotlib import pyplot as plt
#import matplotlib
import numpy as np
import cv2
#import skimage.io
from skimage import io
import copy

#### Configuration
* Set **SHOW_FULLSIZE** = True to show full sized images
* **DELAY** is the duration to show the full sized images in milliseconds

In [2]:
SHOW_FULLSIZE = False
DELAY = 2000                # in milliseconds
INPUT_PATH = '../data/'
OUTPUT_PATH = '../output/'
OUTPUT_THRESHOLD = 64       # Threshold for final output
CROPPING_THRESHOLD = 1      # Threshold used to select the crop outline

fname = 'uoedh_culver house letters12659'

listplug = io.find_available_plugins()
print(listplug)

io.use_plugin(name='imageio', kind='imshow')

{'fits': ['imread', 'imread_collection'], 'gdal': ['imread', 'imread_collection'], 'gtk': ['imshow'], 'imageio': ['imread', 'imsave', 'imread_collection'], 'imread': ['imread', 'imsave', 'imread_collection'], 'matplotlib': ['imshow', 'imread', 'imshow_collection', 'imread_collection'], 'pil': ['imread', 'imsave', 'imread_collection'], 'qt': ['imshow', 'imsave', 'imread', 'imread_collection'], 'simpleitk': ['imread', 'imsave', 'imread_collection'], 'tifffile': ['imread', 'imsave', 'imread_collection']}


RuntimeError: Plugin imageio does not support `imshow`.

### Helper functions

* **makethumb()** is a quick rescaling function, reducing an image to 10% linear size
* **showbriefly()** shows an image for DELAY milliseconds or until a key is pressed
* **showimage()** is a combined embedded and popup display

In [ ]:
def showimage(i):
    io.imshow(makethumb(i), plugin='ioimage')
    showbriefly(i)

def makethumb(i):
    return cv2.resize(i,None,fx=0.1,fy=0.1,interpolation=cv2.INTER_CUBIC)

def showbriefly(i):
    if SHOW_FULLSIZE:
        v = cv2.imshow('Image',i)
        cv2.waitKey(DELAY)
        cv2.destroyWindow(v)

### Load the image
* Image is read as a BGR matrix

In [ ]:
img = cv2.imread(filename=INPUT_PATH+fname+'.jpg')
showimage(img)

#### Convert to grayscale

In [ ]:
grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
showimage(grey)

### Use cv.threshold() to isolate writing
* note that **cv2.threshold()** returns two arguments:
 * **ret** the threshold mask pattern (temporary variable)
 * **thresh** the image with threshold applied
 * Uses the adaptive **OTSU** algorithms for better results on uneven exposures

In [ ]:
ret,thresh = cv2.threshold(grey,OUTPUT_THRESHOLD,255,cv2.THRESH_BINARY)
showimage(thresh)

#### Find the Contours
* Redo the threshold with more extreme settings
* Find the contours of the objects in the image


From: https://stackoverflow.com/questions/13538748/crop-black-edges-with-opencv

In [ ]:
_, cropthresh = cv2.threshold(thresh,CROPPING_THRESHOLD,255,cv2.THRESH_BINARY)

contours,hierarchy = cv2.findContours(cropthresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

#### Find contour with maximal area (largest item on page)
* Step through each of the contours[] keeping track of the contour giving the maximal value
* Store the max contour in **cont**
* *might be a better way of doing this*

In [ ]:
maxc = []
maxa = 0
for c in contours:
    a = cv2.contourArea(c)
    if a>maxa:
        maxc=c
        maxa=a
        print('new maximum: '+str(maxa))
cont = maxc

#### Contour results:


In [ ]:
print(cont)

#print(hierarchy)

#### Find the bounding rectangle of that maximal contour

In [ ]:
x,y,w,h = cv2.boundingRect(cont)

print('x,y,x`,y` = ',x,y,x+w,y+h)
print('w,h = ',w,h)

#### Show the bounding box in green for confirmation
* Note that the **rectangle()** function modifies the image it references
 * Hence the need to do a **deepcopy()**

In [ ]:
boundbox = copy.deepcopy(img)
cv2.rectangle(boundbox,(x,y),(x+w,y+h),(0,255,0),20)
showimage(boundbox)

#### Crop the original image using numpy matrix slicing

In [ ]:
crop = img[y:y+h,x:x+w]
showimage(crop)

#### And now thresholded image

In [ ]:
cropocr = thresh[y:y+h,x:x+w]
showimage(cropocr)

#### Write the output images

In [ ]:
print('Writing cropped colour image')
_ = cv2.imwrite(img=crop,filename=OUTPUT_PATH+fname+'-cropped.jpg')
print('Writing cropped thresholded image for OCR processing')
_ = cv2.imwrite(img=cropocr, filename=OUTPUT_PATH+fname+'-thresholded.jpg')

#### Remove any stray image windows

In [ ]:
cv2.destroyAllWindows()